# Agentbyte Tools: Function Wrapping Examples

This notebook demonstrates how to create tools in Agentbyte using two approaches:
1. **Direct FunctionTool** - Manually wrapping a function
2. **@tool Decorator** - Automatic wrapping with clean syntax

We'll also explore the decorator logic that enables both `@tool` and `@tool(...)` syntaxes.

In [2]:
from agentbyte.tools import FunctionTool, tool, ApprovalMode
from agentbyte.types import ToolResult

## Example 1: Direct FunctionTool Wrapping

## Understanding ToolResult

`ToolResult` is the standardized structure for all tool execution outcomes. It's a Pydantic BaseModel with immutable fields.


In [3]:
# Create ToolResult instances directly

# Success case
success_result = ToolResult(
    success=True,
    result="Operation completed successfully",
    error=None,
    metadata={"tool_name": "example_tool", "execution_time_ms": 125}
)

print("Success ToolResult:")
print(f"  - success: {success_result.success}")
print(f"  - result: {success_result.result}")
print(f"  - error: {success_result.error}")
print(f"  - metadata: {success_result.metadata}")

# Failure case
failure_result = ToolResult(
    success=False,
    result=None,
    error="Database connection timeout",
    metadata={"tool_name": "database_tool", "exception_type": "TimeoutError"}
)

print("\nFailure ToolResult:")
print(f"  - success: {failure_result.success}")
print(f"  - result: {failure_result.result}")
print(f"  - error: {failure_result.error}")
print(f"  - metadata: {failure_result.metadata}")

# ToolResult is immutable - trying to modify raises an error
print("\nTesting immutability:")
try:
    success_result.success = False
    print("  ERROR: ToolResult should be immutable!")
except Exception as e:
    print(f"  ✓ Immutability enforced: {type(e).__name__}")

Success ToolResult:
  - success: True
  - result: Operation completed successfully
  - error: None
  - metadata: {'tool_name': 'example_tool', 'execution_time_ms': 125}

Failure ToolResult:
  - success: False
  - result: None
  - error: Database connection timeout
  - metadata: {'tool_name': 'database_tool', 'exception_type': 'TimeoutError'}

Testing immutability:
  ✓ Immutability enforced: ValidationError


In [4]:
# Define a simple function
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

# Manually wrap it with FunctionTool
add_tool = FunctionTool(
    func=add_numbers,
    name="add",
    description="Adds two integers"
)

print(f"Tool Name: {add_tool.name}")
print(f"Tool Description: {add_tool.description}")
print(f"Parameters Schema: {add_tool.parameters}")

Tool Name: add
Tool Description: Adds two integers
Parameters Schema: {'type': 'object', 'properties': {'a': {'type': 'integer', 'description': "int([x]) -> integer\nint(x, base=10) -> integer\n\nConvert a number or string to an integer, or return 0 if no arguments\nare given.  If x is a number, return x.__int__().  For floating-point\nnumbers, this truncates towards zero.\n\nIf x is not a number or if base is given, then x must be a string,\nbytes, or bytearray instance representing an integer literal in the\ngiven base.  The literal can be preceded by '+' or '-' and be surrounded\nby whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.\nBase 0 means to interpret the base from the string as an integer literal.\n>>> int('0b100', base=0)\n4"}, 'b': {'type': 'integer', 'description': "int([x]) -> integer\nint(x, base=10) -> integer\n\nConvert a number or string to an integer, or return 0 if no arguments\nare given.  If x is a number, return x.__int__().  For floating-point\

## Example 2: @tool Decorator (No Parentheses)

## Example 1b: ToolResult and Error Handling

In [5]:
# ToolResult handles errors gracefully
async def test_error_handling():
    # Invalid parameters - wrong type
    result_invalid = await add_tool.execute({"a": "invalid", "b": 3})
    print("Invalid parameters test:")
    print(f"  Success: {result_invalid.success}")
    print(f"  Error: {result_invalid.error}")
    print(f"  Exception type: {result_invalid.metadata.get('exception_type')}")
    
    # Missing required parameter
    result_missing = await add_tool.execute({"a": 5})
    print("\nMissing required parameter test:")
    print(f"  Success: {result_missing.success}")
    print(f"  Error: {result_missing.error}")

await test_error_handling()

Invalid parameters test:
  Success: False
  Error: Invalid parameters provided
  Exception type: None

Missing required parameter test:
  Success: False
  Error: Invalid parameters provided


In [8]:
# Using @tool without parentheses - uses function name and docstring
@tool
def multiply_numbers(x: int, y: int) -> int:
    """Multiply two numbers together."""
    return x * y

print(f"Tool Name: {multiply_numbers.name}")
print(f"Tool Description: {multiply_numbers.description}")
print(f"Parameters Schema: {multiply_numbers.parameters}")

# Note: Can still call directly!
direct_result = multiply_numbers(4, 5)
print(f"\nDirect call: multiply_numbers(4, 5) = {direct_result}")

Tool Name: multiply_numbers
Tool Description: Multiply two numbers together.
Parameters Schema: {'type': 'object', 'properties': {'x': {'type': 'integer', 'description': "int([x]) -> integer\nint(x, base=10) -> integer\n\nConvert a number or string to an integer, or return 0 if no arguments\nare given.  If x is a number, return x.__int__().  For floating-point\nnumbers, this truncates towards zero.\n\nIf x is not a number or if base is given, then x must be a string,\nbytes, or bytearray instance representing an integer literal in the\ngiven base.  The literal can be preceded by '+' or '-' and be surrounded\nby whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.\nBase 0 means to interpret the base from the string as an integer literal.\n>>> int('0b100', base=0)\n4"}, 'y': {'type': 'integer', 'description': "int([x]) -> integer\nint(x, base=10) -> integer\n\nConvert a number or string to an integer, or return 0 if no arguments\nare given.  If x is a number, return x.__int

In [11]:
# Execute as a tool
async def test_example2():
    result = await multiply_numbers.execute({"x": 4, "y": 5})
    print(f"Tool Execution - Success: {result.success}, Result: {result.result}")

await test_example2()

Tool Execution - Success: True, Result: 20


## Example 3: @tool Decorator (With Parentheses)

In [12]:
# Using @tool with parentheses - custom name and approval mode
# Option 1: Using string (convenient)
@tool(
    name="delete_operation",
    description="Permanently delete data",
    approval_mode="always_require"
)
def delete_data(item_id: int) -> str:
    """Delete an item (requires approval before execution)."""
    return f"Deleted item {item_id}"

print("Option 1: approval_mode as string")
print(f"Tool Name: {delete_data.name}")
print(f"Tool Description: {delete_data.description}")
print(f"Approval Mode: {delete_data.approval_mode}")
print(f"Approval Mode Value: {delete_data.approval_mode.value}")

# Option 2: Using ApprovalMode enum (type-safe)
@tool(
    name="admin_reset",
    description="Reset admin credentials",
    approval_mode=ApprovalMode.ALWAYS
)
def reset_admin(password: str) -> str:
    """Reset admin password (requires approval)."""
    return f"Admin reset with password: {password}"

print("\nOption 2: approval_mode as ApprovalMode enum")
print(f"Tool Name: {reset_admin.name}")
print(f"Approval Mode: {reset_admin.approval_mode}")
print(f"Approval Mode is ALWAYS: {reset_admin.approval_mode == ApprovalMode.ALWAYS}")

Option 1: approval_mode as string
Tool Name: delete_operation
Tool Description: Permanently delete data
Approval Mode: ApprovalMode.ALWAYS
Approval Mode Value: always_require

Option 2: approval_mode as ApprovalMode enum
Tool Name: admin_reset
Approval Mode: ApprovalMode.ALWAYS
Approval Mode is ALWAYS: True


In [13]:
# Execute the tools and inspect ToolResult
async def test_example3():
    # Execute delete_data tool
    result1 = await delete_data.execute({"item_id": 42})
    print("delete_data execution:")
    print(f"  - Type: {type(result1).__name__}")
    print(f"  - Success: {result1.success}")
    print(f"  - Result: {result1.result}")
    print(f"  - Error: {result1.error}")
    print(f"  - Metadata: {result1.metadata}")
    
    # Execute reset_admin tool
    result2 = await reset_admin.execute({"password": "newpass123"})
    print("\nreset_admin execution:")
    print(f"  - Type: {type(result2).__name__}")
    print(f"  - Success: {result2.success}")
    print(f"  - Result: {result2.result}")

await test_example3()

delete_data execution:
  - Type: ToolResult
  - Success: True
  - Result: Deleted item 42
  - Error: None
  - Metadata: {'tool_name': 'delete_operation'}

reset_admin execution:
  - Type: ToolResult
  - Success: True
  - Result: Admin reset with password: newpass123


## The Decorator Logic: How Both Syntaxes Work

### `@tool` (No Parentheses)
```python
@tool              # Python calls: get_weather = tool(get_weather)
def get_weather(): # func is NOT None
    ...
```

- `func` parameter gets the actual function
- Returns `FunctionTool` immediately (1 step)

### `@tool(...)` (With Parentheses)
```python
@tool(approval_mode="...")  # Python calls: decorator_fn = tool(approval_mode="...")
def delete_file():          # Then: delete_file = decorator_fn(delete_file)
    ...
```

**Step 1:** `tool(approval_mode="...", func=None)` returns the inner `decorator` function
- `func` is None
- Returns the `decorator` function object

**Step 2:** Python calls `decorator(delete_file)` with the actual function
- `decorator` creates and returns `FunctionTool`

**Key:** The `decorator` function captures variables from Step 1 (closure), so it "remembers" the `approval_mode` when called in Step 2!

## Summary: Three Ways to Create Tools

| Approach | Usage | When to Use |
|----------|-------|------------|
| **FunctionTool** | `tool = FunctionTool(func, ...)` | Need precise control, runtime creation |
| **@tool** | `@tool` above function | Simple tools with defaults |
| **@tool(...)** | `@tool(name=..., approval_mode=...)` | Custom metadata, approval requirements |

All three approaches create identical `FunctionTool` instances that agents can use.
The decorator versions just provide syntactic convenience!